In [1]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.configs.vits_config import VitsConfig

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
output_path = os.path.dirname(os.path.abspath('__file__'))


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="Mixed_formatted.txt", path=os.path.join(output_path, "data/"), language = "en"
)

In [3]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [4]:
character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890",
    punctuations=""" !,.?-"'""",
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [5]:
config = VitsConfig(
    audio=audio_config,
    # characters=character_config, # Comment out if with phonemes
    run_name="vits_tyler1_phonemes",
    batch_size=4,
    eval_batch_size=2,
    # batch_group_size=4,
    num_loader_workers=8,
    num_eval_loader_workers=2,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=5000,
    text_cleaner="english_cleaners",
    use_phonemes=True, # Replace with False if no phonemes
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path + "/vitstts_checkpoint",
    datasets=[dataset_config],
    cudnn_benchmark=False,
    test_sentences = ["Mic test one two three", "I am Tyler1", "Welcome chat!"]
)

In [6]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "Tyler1"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.dirname(os.path.abspath('__file__')) + f"/data/wavs/{cols[0]}.wav"
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [7]:
ap = AudioProcessor.init_from_config(config)

tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [8]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    # eval_split_max_size=config.eval_split_max_size,
    eval_split_size=0.05,
    formatter = formatter
)

 | > Found 60 files in /home/ubuntu/tyler1/data


In [9]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [10]:
# Make sure to change phoneme in config cell when training a phoneme model

model_path = "vitstts_checkpoint/vits_tyler1-January-16-2024_01+41PM-d5007e2/" # no phonemes
model_path = "vitstts_checkpoint/vits_tyler1-January-17-2024_09+38AM-d5007e2/" # with phonemes

trainer = Trainer(
    # TrainerArgs(restore_path = model_path + "/best_model.pth"), # Load from checkpoint
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 30
 | > Num. of Torch Threads: 30
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
2024-01-23 09:21:01.016354: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 09:21:01.059846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
 > Start Tensorboard: tensorboard --logdir=/h

In [ ]:
trainer.fit()


 > EPOCH: 0/5000
 --> /home/ubuntu/tyler1/vitstts_checkpoint/vits_tyler1_phonemes-January-23-2024_09+21AM-1edd3ad


[*] Pre-computing phonemes...


  4%|▎         | 2/57 [00:00<00:03, 16.74it/s]

ɡˈaɪz oʊkˈeɪ lˈɪsən lˈɛts tˈɔːk ɐbˌaʊt sˈʌmθɪŋ ɹˈiːəl kwˈɪk. ˈaɪ swˈɛɹ tə ɡˈɑːd lˈaɪk jˈɛh maɪ kˈɑːɹdɪˌoʊ wˈʌzn̩t ðə bˈɛst bˌʌt bɹˈoʊ aɪm nˌɑːt lˈaɪk aɪm nˌɑːt kˈoʊp lˈaɪk aɪm nˌɑːt kɹˈeɪzi. ˈaɪ hæv ðɪs ʃˈɑːɹp pˈeɪn ɪn maɪ hˈɑːɹt fɹʌm tˈaɪm tə tˈaɪm. lˈaɪk ˈæktʃuːəl ʃˈɑːɹp, lˈaɪk ɹˈaɪt wɛn ˈaɪ lˈaɪk dˈuː ɐ θˈɜːɾi sˈɛkənd dʒˈɑːɡ ɔːɹ wʌtˈɛvɚɹ ɔːɹ wɛn ˈaɪ snˈiːz ɔːɹ wɛn ˈaɪ dˈuː bˈɛntʃ. ˈaɪ fˈiːl ɪɾ ɐ lˈɑːt. aɪm nˌɑːt kɹˈeɪzi lˈaɪk aɪm nˌɑːt ˈaɪ swˈɛɹ tə ɡˈɑːd ˈæktʃuːəli.
 [!] Character '̩' not found in the vocabulary. Discarding it.


100%|██████████| 57/57 [00:08<00:00,  6.94it/s]

 > TRAINING (2024-01-23 09:21:11) 


...

/home/ubuntu/.local/lib/python3.10/site-packages/TTS/tts/models/vits.py:1455: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  test_figures["{}-alignment".format(idx)] = plot_alignment(alignment.T, output_fig=False)

  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.006366252899169922 (+0)
     | > avg_loss_disc: 3.0608999729156494 (+0)
     | > avg_loss_disc_real_0: 0.29910117387771606 (+0)
     | > avg_loss_disc_real_1: 0.2792060077190399 (+0)
     | > avg_loss_disc_real_2: 0.2866024374961853 (+0)
     | > avg_loss_disc_real_3: 0.26852667331695557 (+0)
     | > avg_loss_disc_real_4: 0.3008735775947571 (+0)
     | > avg_loss_disc_real_5: 0.27346551418304443 (+0)
     | > avg

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.005904436111450195 (-0.00046181678771972656)
     | > avg_loss_disc: 3.1309821605682373 (+0.07008218765258789)
     | > avg_loss_disc_real_0: 0.24920599162578583 (-0.04989518225193024)
     | > avg_loss_disc_real_1: 0.40514233708381653 (+0.1259363293647766)
     | > avg_loss_disc_real_2: 0.3895493447780609 (+0.10294690728187561)
     | > avg_loss_disc_real_3: 0.33496612310409546 (+0.06643944978713989)
     | > avg_loss_disc_real_4: 0.3695238530635834 (+0.0686502754688263)
     | > avg_loss_disc_real_5: 0.35567277669906616 (+0.08220726251602173)
     | > avg_loss_0: 3.1309821605682373 (+0.07008218765258789)
     | > avg_loss_gen: 2.1165528297424316 (+0.3887670040130615)
     | > avg_loss_kl: 4.365316390991211 (-10.797410011291504)
     | > avg_loss_feat: 0.0339265875518322 (-0.14325501397252083)
     | > avg_loss_mel: 74.19326782226562 (+9.887184143066406)
     | > avg_loss_duration: 2.605498790740967 (-0.33777451515197754)
     | > av

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0062410831451416016 (+0.00033664703369140625)
     | > avg_loss_disc: 3.0899081230163574 (-0.04107403755187988)
     | > avg_loss_disc_real_0: 0.2353300005197525 (-0.013875991106033325)
     | > avg_loss_disc_real_1: 0.23704499006271362 (-0.1680973470211029)
     | > avg_loss_disc_real_2: 0.2671225666999817 (-0.12242677807807922)
     | > avg_loss_disc_real_3: 0.32696661353111267 (-0.007999509572982788)
     | > avg_loss_disc_real_4: 0.3918340504169464 (+0.022310197353363037)
     | > avg_loss_disc_real_5: 0.14259199798107147 (-0.2130807787179947)
     | > avg_loss_0: 3.0899081230163574 (-0.04107403755187988)
     | > avg_loss_gen: 1.619883418083191 (-0.4966694116592407)
     | > avg_loss_kl: 2.281982898712158 (-2.0833334922790527)
     | > avg_loss_feat: 0.09966837614774704 (+0.06574178859591484)
     | > avg_loss_mel: 44.18148422241211 (-30.011783599853516)
     | > avg_loss_duration: 2.58008074760437 (-0.02541804313659668)
     | >